In [1]:
# Global configurations and imports
import os
import pandas as pd
import numpy as np
import scipy.io as sio
from matplotlib import pyplot as plt
from nilearn import plotting as niplot

In [2]:
code_directory = '/Users/tereza/nishant/atlas/atlas_work_terez/atlas_harmonization/code'
os.chdir(code_directory)  # Set working directory to code directory

# Paths to Data and Results folders
base_path_results = '../results'
base_path_data = '../Data'

# Paths to data files
desikan_killiany = os.path.abspath(os.path.join(base_path_data, 'aparc+aseg.nii.gz'))
atlas_LUT = os.path.abspath(os.path.join(base_path_data, 'desikanKilliany.csv'))

# Ensure the files exist
assert os.path.exists(desikan_killiany), f"Atlas file not found: {desikan_killiany}"
assert os.path.exists(atlas_LUT), f"LUT file not found: {atlas_LUT}"

In [ ]:
# Function to simplify labels
def simplify_label(label):
    if "Rel" in label:
        return label.split("Rel")[0]  # Simplifies 'deltaRel_mean' to 'delta'
    return label

# Function to get node coordinates from atlas
def get_node_coords(atlas, atlas_LUT):
    coords, labels = niplot.find_parcellation_cut_coords(atlas, return_label_names=True)
    labels = np.array(labels, dtype=np.int64)
    
    atlas_coord = pd.DataFrame({
        'roiNum': labels,
        'mni_x': coords[:, 0], 
        'mni_y': coords[:, 1], 
        'mni_z': coords[:, 2]
    })
    
    atlas_LUT_df = pd.read_csv(atlas_LUT)
    atlas_coord = pd.merge(atlas_LUT_df, atlas_coord, on='roiNum', how='inner')
    atlas_coord.to_csv(os.path.join(base_path_results, 'atlas_coord.csv'), index=False)
    return atlas_coord

# Load and process data
print(f"Using atlas path: {desikan_killiany}")
print(f"Using LUT path: {atlas_LUT}")
atlas_coord = get_node_coords(atlas=desikan_killiany, atlas_LUT=atlas_LUT)

# Interactive Visualization with simplified labels
atlas_coord['simplified_abvr'] = atlas_coord['abvr'].apply(simplify_label)
view = niplot.view_markers(
    atlas_coord[['mni_x', 'mni_y', 'mni_z']],  # Coordinates
    marker_color='red',
    marker_size=10,
    marker_labels=atlas_coord['simplified_abvr'].tolist()  # Simplified region abbreviations
)
view.open_in_browser()  # Launch interactive visualization

# Example of loading additional data and merging
hup_atlas = sio.loadmat(os.path.join(base_path_data, 'HUP_atlas.mat'))
hup_df = pd.read_csv(os.path.join(base_path_data, 'hup_df.csv'))

# Handling of mni_coords
mni_coords = hup_atlas['mni_coords']
if len(mni_coords.shape) > 2:
    mni_coords = mni_coords.reshape(-1, 3)

mni_coords_df = pd.DataFrame(mni_coords, columns=['mni_x', 'mni_y', 'mni_z'])
hup_atlas_with_roiNum = pd.concat([mni_coords_df, hup_df.reset_index(drop=True)], axis=1)
output_path = os.path.join(base_path_results, 'hup_atlas_with_roiNum.csv')
hup_atlas_with_roiNum.to_csv(output_path, index=False)
print(f"Concatenated DataFrame saved to {output_path}")

auc_data = []
for region_name, features in results.items():
    roi_num = region_to_roiNum.get(region_name)
    if roi_num is not None:
        auc_value = features.get('deltaRel_mean', {}).get('AUC', None)
        if auc_value is not None:
            auc_data.append({'roiNum': roi_num, 'AUC': auc_value})

auc_df = pd.DataFrame(auc_data)
print("AUC DataFrame:", auc_df.head())

# Merging and Visualization continued...


Using atlas path: /Users/tereza/nishant/atlas/atlas_work_terez/atlas_harmonization/Data/aparc+aseg.nii.gz
Using LUT path: /Users/tereza/nishant/atlas/atlas_work_terez/atlas_harmonization/Data/desikanKilliany.csv
Concatenated DataFrame saved to ../results/hup_atlas_with_roiNum.csv


NameError: name 'results' is not defined